In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


## ДЗ

Постройте топ фильмов в категориях Action и Comedy

In [7]:
ratings_grouped = ratings.groupby('movieId')['rating'].mean()
ratings_stdev = ratings.groupby('movieId')['rating'].agg(np.std)
ratings_count = ratings.groupby('movieId')['rating'].count()

In [8]:
mean_rate=ratings['rating'].mean()
mean_rate

3.501556983616962

In [9]:
movies = movies.join(ratings_grouped, on = 'movieId').rename(columns = {'rating':'avg_rating'})
movies = movies.join(ratings_stdev, on = 'movieId').rename(columns = {'rating':'stdev'})
movies = movies.join(ratings_count, on = 'movieId').rename(columns = {'rating':'amount'})
movies.head(3)

,movieId,title,genres,avg_rating,stdev,amount
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,0.834859,215.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,0.881713,110.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,1.054823,52.0


In [10]:
movies['avg_rating'] = movies['avg_rating'] - mean_rate

In [11]:
mean_nrate = movies['amount'].mean()
mean_nrate

10.369806663924312

In [12]:
movies['amount'] = movies['amount'] - mean_nrate

In [13]:
movies.head(3)

,movieId,title,genres,avg_rating,stdev,amount
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.419373,0.834859,204.630193
1,2,Jumanji (1995),Adventure|Children|Fantasy,-0.069739,0.881713,99.630193
2,3,Grumpier Old Men (1995),Comedy|Romance,-0.241942,1.054823,41.630193


Расчитал статистики оценок для расчета score фильмов. Решил нормировать средний рейтинг и ограничить средним количество оценок за фильм

In [14]:
movies['score']=(movies['avg_rating']/movies['stdev'])*movies['amount']

In [15]:
movies.drop(movies[movies['amount'] <= 0].index, inplace = True)

movies['score'] = movies['score'] - movies['score'].min()

In [16]:
movies.sort_values('score', ascending = True).head()

,movieId,title,genres,avg_rating,stdev,amount,score
126,153,Batman Forever (1995),Action|Adventure|Comedy|Crime,-0.585499,0.904583,126.630193,0.000000
302,344,Ace Ventura: Pet Detective (1994),Comedy,-0.461184,1.070886,150.630193,17.092546
176,208,Waterworld (1995),Action|Adventure|Sci-Fi,-0.588514,0.955733,104.630193,17.534089
18,19,Ace Ventura: When Nature Calls (1995),Comedy,-0.774284,1.063796,77.630193,25.459272
2029,2701,Wild Wild West (1999),Action|Comedy|Sci-Fi|Western,-1.294010,0.982706,42.630193,25.827766


Пробую вывести топ фильмов содержащих 'Action' в поле жанр

In [17]:
action_top = movies[movies['genres'].str.contains('Action')].sort_values('score', ascending=False)
action_top.head(10)

,movieId,title,genres,avg_rating,stdev,amount,score
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,0.729519,0.872004,240.630193,283.273782
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,0.690889,0.975243,267.630193,271.559155
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller,0.771379,0.861384,207.630193,267.897473
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,0.714083,0.842158,200.630193,252.080810
900,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,0.705943,0.794576,189.630193,250.439929
1503,2028,Saving Private Ryan (1998),Action|Drama|War,0.644720,0.811770,177.630193,223.038998
6710,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,0.736698,0.724997,138.630193,222.829971
911,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,0.636198,0.866116,185.630193,218.315514
4800,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,0.617362,0.849820,174.630193,208.824605
97,110,Braveheart (1995),Action|Drama|War,0.530089,0.967726,226.630193,206.102990


Кажется, вышло правдоподобно.
Пробуем аналогичное действие для жанра 'Comey'

In [18]:
comedy_top = movies[movies['genres'].str.contains('Comedy')].sort_values('score', ascending=False)
comedy_top.head(10)

,movieId,title,genres,avg_rating,stdev,amount,score
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,0.662577,0.831244,318.630193,335.939715
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0.695511,0.951997,296.630193,298.674950
520,608,Fargo (1996),Comedy|Crime|Drama|Thriller,0.614465,0.828531,170.630193,208.507216
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,0.730837,0.813586,131.630193,200.204754
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.419373,0.834859,204.630193,184.753951
969,1270,Back to the Future (1985),Adventure|Comedy|Sci-Fi,0.536455,0.874484,160.630193,180.501444
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy,0.660208,0.906524,125.630193,173.456974
3622,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,0.681776,0.886247,109.630193,166.299263
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,0.766484,0.807071,86.630193,164.236083
4360,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,0.459436,0.774760,130.630193,159.426681


В списке довольно мало классических комедий - в основном популярные жанровые фильмы с включением нескольких комедийных сцен. Пробуем почистить список исключив фидьмы с жанром 'Drama', 'Adventure' и 'Animation' 

In [19]:
comedy_top.drop(comedy_top[comedy_top['genres'].str.contains('Drama') == True].index, inplace = True)
comedy_top.drop(comedy_top[comedy_top['genres'].str.contains('Animation') == True].index, inplace = True)
comedy_top.drop(comedy_top[comedy_top['genres'].str.contains('Adventure') == True].index, inplace = True)

comedy_top.head(10)

,movieId,title,genres,avg_rating,stdev,amount,score
3622,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,0.681776,0.886247,109.630193,166.299263
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,0.766484,0.807071,86.630193,164.236083
964,1265,Groundhog Day (1993),Comedy|Fantasy|Romance,0.442499,0.849824,132.630193,151.022324
2996,4011,Snatch (2000),Comedy|Crime|Thriller,0.654357,0.793926,82.630193,150.066613
1883,2502,Office Space (1999),Comedy|Crime,0.588869,0.809762,83.630193,142.779347
815,1073,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,0.372393,0.921077,108.630193,125.881754
2195,2918,Ferris Bueller's Day Off (1986),Comedy,0.337893,0.833002,98.630193,121.970043
987,1288,This Is Spinal Tap (1984),Comedy,0.513595,0.754319,55.630193,119.839457
5335,8874,Shaun of the Dead (2004),Comedy|Horror,0.504937,0.930102,66.630193,118.134859
1298,1732,"Big Lebowski, The (1998)",Comedy|Crime,0.422971,1.137651,95.630193,117.517135


Так гораздо лучше